In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class EnhancedCNNModel(nn.Module):
    def __init__(self):
        super(EnhancedCNNModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(256 * 4 * 4, 1024)
        self.drop = nn.Dropout2d(0.5)
        self.fc2 = nn.Linear(1024, 100)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(-1, 256 * 4 * 4)
        x = self.drop(nn.functional.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

model = EnhancedCNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 30
for epoch in range(epochs):
    model.train()
    for batch_idx, (imgs, lbls) in enumerate(train_loader):
        preds = model(imgs)
        loss = criterion(preds, lbls)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

model.eval()
correct_preds = 0
total_preds = 0
with torch.no_grad():
    for imgs, lbls in test_loader:
        preds = model(imgs)
        _, predicted_classes = torch.max(preds.data, 1)
        total_preds += lbls.size(0)
        correct_preds += (predicted_classes == lbls).sum().item()

print(f'Testing Accuracy: {100 * correct_preds / total_preds:.2f}%')


100%|██████████| 169M/169M [00:05<00:00, 29.1MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch [1/30], Loss: 3.7648
Epoch [2/30], Loss: 4.1017
Epoch [3/30], Loss: 3.2849
Epoch [4/30], Loss: 3.4641
Epoch [5/30], Loss: 2.7641
Epoch [6/30], Loss: 2.7722
Epoch [7/30], Loss: 2.7244
Epoch [8/30], Loss: 2.6918
Epoch [9/30], Loss: 3.2282
Epoch [10/30], Loss: 3.5011
Epoch [11/30], Loss: 2.2638
Epoch [12/30], Loss: 2.5820
Epoch [13/30], Loss: 1.8523
Epoch [14/30], Loss: 2.1355
Epoch [15/30], Loss: 3.4551
Epoch [16/30], Loss: 2.9071
Epoch [17/30], Loss: 1.2920
Epoch [18/30], Loss: 2.1745
Epoch [19/30], Loss: 1.4825
Epoch [20/30], Loss: 2.7610
Epoch [21/30], Loss: 1.8341
Epoch [22/30], Loss: 2.2462
Epoch [23/30], Loss: 1.8259
Epoch [24/30], Loss: 2.4284
Epoch [25/30], Loss: 2.6584
Epoch [26/30], Loss: 2.5547
Epoch [27/30], Loss: 2.7133
Epoch [28/30], Loss: 1.6603
Epoch [29/30], Loss: 2.5307
Epoch [30/30], Loss: 1.9871
Testing Accuracy: 49.93%
